In [ ]:



# Before running the tool define the command to run the simulation in cell 15


import json
import simtool as st
from simtool import DB
import uuid
import os
import shutil
import nbformat # type: ignore 
from ipywidgets import HBox, VBox, HTML, Image, Layout, Button, ButtonStyle, Tab, BoundedIntText, BoundedFloatText, Dropdown, ToggleButton, Output, Textarea, FloatLogSlider # type: ignore

print(os.getcwd())
import sys
# Add the parent directory to the sys.path at position 1
sys.path.insert(1, '../../')

# Now you can import the module from the parent directory
# This is needed because the simtool notebook is run inside a run directory two levels down from the output keys file
import outputkeys






%load_ext yamlmagic

/Users/austintoro/Documents/Coding/N-DEVELOP/DEVELOP/Sim2L


In [11]:
# Set debug mode: 1 to enable, 0 to disable
# Debug mode will enable specific print statements and will only adjust the temperature input 

debug_mode = 1

In [12]:
%%yaml INPUTS
Energy_Band_Model: # not needed
    type: Choice
    description: Energy Band Model
    options: ['Effective Mass Model', 'Tight-Binding Sp3s* Model', 'Tight-Binding sp3d5s* Model']
    value: Effective Mass Model

Potential_model:
    type: Choice
    description: Potential Model
    options: ['Thomas-Fermi', 'Hartree']
    value: Thomas-Fermi

Ending_Bias:
    type: Number
    description: Ending Bias
    max: 1000
    min: 0
    value: 0

Device:
    type: Choice
    description: Device Type
    options: ['1-Barrier-Device', '2-Barrier-Device', '3-Barrier-Device', '6-Barrier-Device']
    value: 2-Barrier-Device

points:
    type: Number
    description: No. of Points
    max: 100.0
    min: 1.0
    value: 6

Starting_bias:
    type: Number
    description: Starting Bias
    max: 100.0
    min: 0.0
    value: 0

TEMPERATURE:
    type: Number
    description: Temperature
    max: 5500.0
    min: 77.0


SC_ChargeRegion: #min and max any place with regions
    type: Number
    description: Semiclassical Charge Region dSC (nm)
    min: 0
    value: 0

EQ_Region:
    type: Number
    description: Equilibrium Region dEQ (nm)
    min: 0
    value: 0

Quantum_charge:
    type: Boolean
    description: Quantum Charge
    value: false

Lattice_Constant:
    type: Number
    description: Lattice Constant (nm)
    value: 0.283

Not_Normalized_Current_Plot:
    type: Boolean
    description: Not-Normalized Current Plot
    value: false

Scatter_Plot:
    type: Boolean
    description: Scatter Plot
    value: false

Carrier_Surface_Distrubution_Plot:
    type: Boolean
    description: Carrier Surface Distribution Plot
    value: false

Resivoir_Relaxation_Model:
    type: Choice
    description: Reservoir Relaxation Model
    options: ['Energy Independant', 'Exponentialy Damped', 'Lorentzian Damped']
    value: Energy Independant

Resivoir_Relaxation_Energy:
    type: Number
    description: Reservoir Relaxation Energy (meV)
    min: 0
    value: 0

Decay_Length:
    type: Number
    description: Decay Length (meV)
    min: 0
    value: 0

Resonance_Finder:
    type: Boolean
    description: Resonance Finder
    value: false

No_Homogenous_Grid_Points:
    type: Number
    description: No. Homogeneous Grid Points
    min: 0
    value: 0

No_Points_per_res:
    type: Number
    description: No. Points Per Resonance
    min: 0
    value: 0

No_Points_per_EC_EF:
    type: Number
    description: No. Points Per EC EF
    min: 0
    value: 0

Lancos_Iteration_Step_Size:
    type: Number
    description: Lanczos Iteration Step Size (eV)
    min: 0
    value: 0

Lancos_Iteration_Limit:
    type: Number
    description: Lanczos Iteration Limit
    min: 0
    value: 0

Newton_Iteration_Step_Size:
    type: Number
    description: Newton Iteration Step Size (eV)
    min: 0
    value: 0

Newton_Solver_Convergence_Condition:
    type: Number
    description: Newton Solver Convergence Condition (eV)
    value: 0


<IPython.core.display.Javascript object>

In [13]:
# From the sim2l library 
# Ensures that the values passed are within specified range 
# Uses the default inputs if nothing is passed



parameters = st.utils.getValidatedInputs(INPUTS)  #type= ignore 


if parameters:
    globals().update(parameters)

#print(parameters)
    

In [14]:
# Takes injected parameters and converts it to a dictionary 
# looks through the notebook for a cell with tag injected-paramaters
# Once found it converts the injected parameters to a dictionary and updates the parameters dictionary

def convert_parameters_to_dict(tag):
    # Get the current directory
    current_directory = os.getcwd()
    
    # Define the notebook file name
    notebook_file = os.path.join(current_directory, 'nemo1dsim2l.ipynb')

    # Read the notebook file using nbformat 
    try:
        with open(notebook_file, 'r', encoding='utf-8') as f:
            notebook = nbformat.read(f, as_version=4)
    except Exception as e:
        print(f"Error reading notebook file: {e}")
        return {}

    # Initialize an empty dictionary to hold the parameters
    parameters = {}

    # Loop through each cell in the notebook
    for cell in notebook.cells:
        # Check if the cell has metadata and tags field
        if 'metadata' in cell and 'tags' in cell.metadata:
            # If the tag is in the cell's metadata, process the content
            if tag in cell.metadata['tags']:
                print(f"Found cell with tag '{tag}':")
                print(cell.source)  # Print the content of the cell for debugging
                
                # Extract the content of the cell
                cell_content = cell.source
                
                # Split the content into lines and parse each line
                for line in cell_content.split('\n'):
                    if '=' in line:
                        # Split each line into variable name and value
                        var_name, var_value = line.split('=', 1)
                        var_name = var_name.strip()
                        var_value = var_value.strip()
                        
                        # Attempt to parse the value as an integer, float, or boolean
                        if var_value.lower() in ['true', 'false']:
                            parameters[var_name] = var_value.lower() == 'true'
                        elif var_value.replace('.', '', 1).isdigit():  # Check if it's a float
                            parameters[var_name] = float(var_value)
                        else:
                            parameters[var_name] = var_value.strip('"')  # Remove quotes for strings

    return parameters
parameters = convert_parameters_to_dict('injected-parameters')


In [ ]:
%%yaml OUTPUTS

NEMO1D Input Deck:
 type: File    
 description : NEMO1D Input Deck
WSQR:
  type: Dict
  description: WSQR output file

BND:
  type: Dict
  description: BND output file

CHG:
  type: Dict
  description: CHG output file

CHGTOT:
  type: Dict
  description: CHGTOT output file

CURD:
  type: Dict
  description: CURD output file

IV:
  type: Dict
  description: IV output file

J_INT_E:
  type: Dict
  description: J_INT_E output file

J_INT_E_AV:
  type: Dict
  description: J_INT_E_AV output file

J_INT_K:
  type: Dict
  description: J_INT_K output file

J_K:
  type: Dict
  description: J_K output file

J_PHI:
  type: Dict
  description: J_PHI output file

KJ_INT_K:
  type: Dict
  description: KJ_INT_K output file

KJ_K:
  type: Dict
  description: KJ_K output file

KN_XK_INT_K:
  type: Dict
  description: KN_XK_INT_K output file

N_SHEET:
  type: Dict
  description: N_SHEET output file

N_XK:
  type: Dict
  description: N_XK output file

N_XK_INT_K:
  type: Dict
  description: N_XK_INT_K output file

RES:
  type: Dict
  description: RES output file

RESENERGY_V:
  type: Dict
  description: RESENERGY_V output file

RESTRANS_E:
  type: Dict
  description: RESTRANS_E output file

RESTRANS_V:
  type: Dict
  description: RESTRANS_V output file

RESWIDTH_V:
  type: Dict
  description: RESWIDTH_V output file

TRAN:
  type: Dict
  description: TRAN output file

WAVE:
  type: Dict
  description: WAVE output file

    

UsageError: Cell magic `%%yaml` not found.


In [ ]:
# Creates the data base for the outputs 
# Creates a unique identifer for each newinput deck

db = DB(OUTPUTS)  #type: ignore


Newinputdeck= str(uuid.uuid4().hex)

NameError: name 'DB' is not defined

In [ ]:
# Debug Print to show if Debug mode is active

if debug_mode == 1:
    print("Debug Mode Active")

print("Parameters before input deck creation:", parameters)

<class 'str'>
Debug Mode Active
Parameters before input deck creation: {}


In [ ]:
# Copies the desired material file to the Directory where the sim2l is run
# Takes the destination directory as an input 
# The material file is defined in the outputkeys file

def copy_material_file(destination_dir):
  
    dest_material_file = os.path.join(destination_dir, os.path.basename(outputkeys.material_file))
  
    if not os.path.exists(dest_material_file):
        shutil.copy2(outputkeys.material_file, dest_material_file)
        print(f"Copied {outputkeys.material_file} to {destination_dir}")
  
    else:
        print(f"{outputkeys.material_file} already exists in {destination_dir}")

In [ ]:
# Takes the input deck file given in the argument and reads it 

def read_input_deck(filepath):
    
    with open(filepath, 'r') as file:
        return file.read()
    



'<begin_header>\nwhat = "@(#)NEMO Input File -- Version 4.0.2"\nftype = Nemo_Input_File \nversion = "4.0.2"\n%\t\t\tcomments = "Some Character String"\n<end_header>\n\n<begin_file_list>\nInput = "st.nem"\n% MatParaDir = "/home/gekco/nemo/param_4.0.2/"\nMatParaDir = "./"\n%\t\t\tDataFiles = "Some Character String"\n<end_file_list>\n\n\n<begin_layer_list>\n\n<begin_layer>\nlayer_name = "L0"\nmaterial = "GaAs"\n%\t\t\tcomp1_mat = "Some Character String"\nx_mole_frac = 0 \nx_mole_frac_end = 0 \n%\t\t\tcomp2_mat = "Some Character String"\ny_mole_frac = 0 \ndoping = 1e+18 \ndoping_end = 0 \nfermi_level_fix = 0 \nthickness = 30.0298 \nmonolayers = 106 \nroughness = 0 \nterminal_no = 1 \n<end_layer>\n\n<begin_layer>\nlayer_name = "L1"\nmaterial = "GaAs"\n%\t\t\tcomp1_mat = "Some Character String"\nx_mole_frac = 0 \nx_mole_frac_end = 0 \n%\t\t\tcomp2_mat = "Some Character String"\ny_mole_frac = 0 \ndoping = 2e+15 \ndoping_end = 0 \nfermi_level_fix = 0 \nthickness = 30.0298 \nmonolayers = 106 \n

In [ ]:
# Replaces the values from parameters and puts them in the new input deck
# Debug mode makes it to wher only temperature is changed 
# This function looks through the input deck for specific keys and replaces them with the values from the parameters dictionary
# They keys are defined in Parameters dictionary


debug_mode == 1
print(debug_mode)
def replace_placeholders(content, parameters):
    ""
    if debug_mode == 1:
        # Only replace the TEMPERATURE placeholder
        key = "TEMPERATURE"
        if key in parameters:
            content = content.replace(key, str(parameters[key]))
            print(f"Replacing {key} with {parameters[key]}")
    else:
        # Replace all placeholders
        for key, value in parameters.items():
            content = content.replace(key, str(value))
            print(f"Replacing {key} with {value}")
    return content

1


In [28]:
# Writes the new input deck with the values from the dictionary
# Takes the new paraamters new_content and output file name as arguments 
# Returns the input deck with injected parameters

def write_new_input_deck(new_content, output_filename):
    with open(output_filename, 'w') as file:
        file.write(new_content)
    print(f"New input deck written to {output_filename}")
    return output_filename

In [ ]:
# Copies all the required files and returns a new input deck 
# This function combines other functions to generate a new input deck based on the injected parameters
# Returns the new input deck




def inputdeckCreator():
 
    current_dir = os.getcwd()

    copy_material_file(current_dir)

    content = read_input_deck(outputkeys.inputdeck) 

    updated_content = replace_placeholders(content, parameters)
    
    

    write_new_input_deck(updated_content, outputkeys.new_input_deck_filename)

    if debug_mode == 1:
        print(f"Debug mode: Simulating with input deck {outputkeys.new_input_deck_filename}")

    return outputkeys.new_input_deck_filename


In [36]:
inputdeckCreator()
db.save("NEMO1D Input Deck", file='new_input_deck.nem') 




/Users/austintoro/Documents/Coding/NEMO1D-Local_Simtool/GaAs.mat already exists in /Users/austintoro/Documents/Coding/NEMO1D-Local_Simtool/simtool
New input deck written to new_input_deck.nem
Debug mode: Simulating with input deck new_input_deck.nem


In [ ]:
# Runs the NEMO1D simulation with the new input deck

!/Users/austintoro/Documents/Coding/NEMO1D-Local_Simtool/simtool/Negf new_input_deck.nem

system_start WARNING: The NEMOHOME environmental variable is not set
NEMO will assume "/usr/local/nemo" is the correct home directory.
No memory report in this platform.
Open File: new_input_deck.nem, 
Could not find the Attribute
"AlGaAs.BandModel.Cgam.unstrnd_cubic_cell_length"
in the database. 
Needed for NEMO 3-D but not for NEMO 1-D
Return a zero value.
Could not find the Attribute
"AlGaAs.PhysChem.strain_alpha"
in the database. 
Needed for NEMO 3-D but not for NEMO 1-D
Return a zero value.
Could not find the Attribute
"AlGaAs.PhysChem.strain_beta"
in the database. 
Needed for NEMO 3-D but not for NEMO 1-D
Return a zero value.
Initialize the basic device structure arrays.
Update Structural Device Parameters and Initialize Arrays
Copy the material information to the individual monolayers.
Copy Material Parameters into Device Arrays
Finished Device Structure Update
Process Simulation Parameter Window: Calculation Mode
Read and Initialize Simulation Parameter Window: I-V Parameters
P

In [ ]:
#
# The simulation ouputs JSON files, this function loops through the paths dictionary in the outputkeys file, and saves the files to the sim2l database
# The processing of the data takes place in the workflow




for name, data in outputkeys.paths.items():
    try:
        db.save(name, file=data["filename"])
    except FileNotFoundError:
        print(f"File not found: {name}. Skipping...")
    except Exception as e:
        print(f"An error occurred while saving {name}: {e}. Skipping...")

FileNotFoundError: [Errno 2] No such file or directory: 'new_input_deck.nd_wsqr.json'